In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from utils import *

In [3]:
train = pd.read_csv('train_dataset.csv')
test = pd.read_csv('test_dataset.csv')

In [4]:
sample = pd.read_csv('sample_submission.csv')

In [5]:
sess = tf.InteractiveSession()

In [6]:
batch_size = 10

In [ ]:
insample_evaluates = []
outsample_evaluates = []
com_cats = []
for rid, com_range in enumerate(full_ranges[1:]):
    print(rid)
    col_ids = list_sum(x[1] for x in com_range)
    com_cat = string_sum(x[0] for x in com_range)
    x = tf.placeholder('float', shape=[None, len(col_ids)])
    y_ = tf.placeholder('float', shape=[None, 3])
    l2 = tf.placeholder('float')
    hidden1 = 2000
    hidden2 = 100
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
            tf.truncated_normal([len(col_ids), hidden1],
                                stddev=1.0 / np.sqrt(float(len(col_ids)))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden1]),
                             name='biases')
        hidden1_tensor = tf.nn.relu(tf.matmul(x, weights) + biases)
        reg_hidden1 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal([hidden1, hidden2],
                                stddev=1.0 / np.sqrt(float(hidden1))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden2]),
                             name='biases')
        hidden2_tensor = tf.nn.relu(tf.matmul(hidden1_tensor, weights) + biases)
        reg_hidden2 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden2, 3],
                                stddev=1.0 / np.sqrt(float(hidden2))),
            name='weights')
        biases = tf.Variable(tf.zeros([3]),
                             name='biases')
        logits = tf.matmul(hidden2_tensor, weights) + biases
        reg_hidden3 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    sess.run(tf.initialize_all_variables()) 
    y = tf.nn.softmax(logits)
    cross_entropy = - tf.reduce_mean(y_ * tf.log(y) + 1e-15) + l2 * (reg_hidden1 + reg_hidden2 + reg_hidden3)
    train_step = tf.train.GradientDescentOptimizer(.001).minimize(cross_entropy)
    train_x, valid_x, train_y, valid_y = train_test_split(
        train.iloc[:, col_ids],
        train.loc[:, 'fault_severity'],
        test_size=0.3333,
        random_state=100383)
    labels = tf.expand_dims(np.asarray(train_y.values, dtype='int32'), 1)
    indices = tf.expand_dims(tf.range(0, train_x.shape[0]), 1)
    concated = tf.concat(1, [indices, labels])
    train_onehot_labels = tf.sparse_to_dense(
        concated, tf.pack([train_x.shape[0], sample.shape[1]-1]), 1.0, 0.0).eval()
    for i in range(train_x.shape[0]//batch_size):
        train_step.run(
            feed_dict={
                x: train_x.iloc[batch_size*i:batch_size*(i+1), :].values,
                y_: train_onehot_labels[batch_size*i:batch_size*(i+1), :],
                l2: 0})
    labels = tf.expand_dims(np.asarray(valid_y.values, dtype='int32'), 1)
    indices = tf.expand_dims(tf.range(0, valid_x.shape[0]), 1)
    concated = tf.concat(1, [indices, labels])
    valid_onehot_labels = tf.sparse_to_dense(
        concated, tf.pack([valid_x.shape[0], sample.shape[1]-1]), 1.0, 0.0).eval()
    evaluate = - tf.reduce_mean(tf.reduce_sum(y_ * tf.log(y), 1))
    insample_evaluate = evaluate.eval(feed_dict={x: train_x, y_: train_onehot_labels})
    insample_evaluates.append(insample_evaluate)
    outsample_evaluate = evaluate.eval(feed_dict={x: valid_x, y_: valid_onehot_labels})
    outsample_evaluates.append(outsample_evaluate)
    com_cats.append(com_cat)
res = pd.DataFrame(
    {'insample_evaluates': insample_evaluates,
     'outsample_evaluates': outsample_evaluates}, index=com_cats)

In [ ]:
plt.figure(figsize=(10, 7))
ax = plt.subplot(111)
res.loc[:, 'insample_evaluates'].plot(kind='line', ax=ax, rot=90, marker='*')
res.loc[:, 'outsample_evaluates'].plot(kind='line', ax=ax, color='r', rot=90, marker='*')
plt.xticks(range(len(res.index)), res.index)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
ax = plt.subplot(111)
res.loc[:, 'insample_evaluates'].plot(kind='line', ax=ax, rot=90, marker='*')
res.loc[:, 'outsample_evaluates'].plot(kind='line', ax=ax, color='r', rot=90, marker='*')
plt.xticks(range(len(res.index)), res.index)
plt.grid()
plt.show()

In [ ]:
insample_evaluates = []
outsample_evaluates = []
regs = [0.01, .1, .2, .3, .4]
for rid, reg in enumerate(regs):
    print(rid)
    col_ids = list_sum(x[1] for x in ranges[2:4])
    x = tf.placeholder('float', shape=[None, len(col_ids)])
    y_ = tf.placeholder('float', shape=[None, 3])
    l2 = tf.placeholder('float')
    hidden1 = 2000
    hidden2 = 100
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
            tf.truncated_normal([len(col_ids), hidden1],
                                stddev=1.0 / np.sqrt(float(len(col_ids)))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden1]),
                             name='biases')
        hidden1_tensor = tf.nn.relu(tf.matmul(x, weights) + biases)
        reg_hidden1 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal([hidden1, hidden2],
                                stddev=1.0 / np.sqrt(float(hidden1))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden2]),
                             name='biases')
        hidden2_tensor = tf.nn.relu(tf.matmul(hidden1_tensor, weights) + biases)
        reg_hidden2 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden2, 3],
                                stddev=1.0 / np.sqrt(float(hidden2))),
            name='weights')
        biases = tf.Variable(tf.zeros([3]),
                             name='biases')
        logits = tf.matmul(hidden2_tensor, weights) + biases
        reg_hidden3 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    sess.run(tf.initialize_all_variables()) 
    y = tf.nn.softmax(logits)
    cross_entropy = - tf.reduce_mean(y_ * tf.log(y) + 1e-15) + l2 * (reg_hidden1 + reg_hidden2 + reg_hidden3)
    train_step = tf.train.GradientDescentOptimizer(.001).minimize(cross_entropy)
    train_x, valid_x, train_y, valid_y = train_test_split(
        train.iloc[:, col_ids],
        train.loc[:, 'fault_severity'],
        test_size=0.3333)
    labels = tf.expand_dims(np.asarray(train_y.values, dtype='int32'), 1)
    indices = tf.expand_dims(tf.range(0, train_x.shape[0]), 1)
    concated = tf.concat(1, [indices, labels])
    train_onehot_labels = tf.sparse_to_dense(
        concated, tf.pack([train_x.shape[0], sample.shape[1]-1]), 1.0, 0.0).eval()
    for i in range(train_x.shape[0]//batch_size):
        train_step.run(
            feed_dict={
                x: train_x.iloc[batch_size*i:batch_size*(i+1), :].values,
                y_: train_onehot_labels[batch_size*i:batch_size*(i+1), :],
                l2: 0})
    labels = tf.expand_dims(np.asarray(valid_y.values, dtype='int32'), 1)
    indices = tf.expand_dims(tf.range(0, valid_x.shape[0]), 1)
    concated = tf.concat(1, [indices, labels])
    valid_onehot_labels = tf.sparse_to_dense(
        concated, tf.pack([valid_x.shape[0], sample.shape[1]-1]), 1.0, 0.0).eval()
    evaluate = - tf.reduce_mean(tf.reduce_sum(y_ * tf.log(y), 1))
    insample_evaluate = evaluate.eval(feed_dict={x: train_x, y_: train_onehot_labels})
    insample_evaluates.append(insample_evaluate)
    outsample_evaluate = evaluate.eval(feed_dict={x: valid_x, y_: valid_onehot_labels})
    outsample_evaluates.append(outsample_evaluate)
res = pd.DataFrame(
    {'insample_evaluates': insample_evaluates,
     'outsample_evaluates': outsample_evaluates}, index=regs)

In [ ]:
res

In [ ]:
res

In [8]:
col_ids = list_sum(x[1] for x in ranges[2:4])
x = tf.placeholder('float', shape=[None, len(col_ids)])
y_ = tf.placeholder('float', shape=[None, 3])
l2 = tf.placeholder('float')
hidden1 = 2000
hidden2 = 100
with tf.name_scope('hidden1'):
    weights = tf.Variable(
        tf.truncated_normal([len(col_ids), hidden1],
                            stddev=1.0 / np.sqrt(float(len(col_ids)))),
        name='weights')
    biases = tf.Variable(tf.zeros([hidden1]),
                         name='biases')
    hidden1_tensor = tf.nn.relu(tf.matmul(x, weights) + biases)
    reg_hidden1 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
with tf.name_scope('hidden2'):
    weights = tf.Variable(
        tf.truncated_normal([hidden1, hidden2],
                            stddev=1.0 / np.sqrt(float(hidden1))),
        name='weights')
    biases = tf.Variable(tf.zeros([hidden2]),
                         name='biases')
    hidden2_tensor = tf.nn.relu(tf.matmul(hidden1_tensor, weights) + biases)
    reg_hidden2 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
with tf.name_scope('softmax_linear'):
    weights = tf.Variable(
        tf.truncated_normal([hidden2, 3],
                            stddev=1.0 / np.sqrt(float(hidden2))),
        name='weights')
    biases = tf.Variable(tf.zeros([3]),
                         name='biases')
    logits = tf.matmul(hidden2_tensor, weights) + biases
    reg_hidden3 = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
sess.run(tf.initialize_all_variables()) 
y = tf.nn.softmax(logits)
cross_entropy = - tf.reduce_mean(y_ * tf.log(y) + 1e-15) + l2 * (reg_hidden1 + reg_hidden2 + reg_hidden3)
train_step = tf.train.GradientDescentOptimizer(.001).minimize(cross_entropy)
train_x, valid_x, train_y, valid_y = train_test_split(
        train.iloc[:, col_ids],
        train.loc[:, 'fault_severity'],
        test_size=0.3333)
labels = tf.expand_dims(np.asarray(train.loc[:, 'fault_severity'].values, dtype='int32'), 1)
indices = tf.expand_dims(tf.range(0, train.shape[0]), 1)
concated = tf.concat(1, [indices, labels])
train_onehot_labels = tf.sparse_to_dense(
    concated, tf.pack([train.shape[0], sample.shape[1]-1]), 1.0, 0.0).eval()
for i in range(train_x.shape[0]//batch_size):
    train_step.run(
        feed_dict={
            x: train.iloc[batch_size*i:batch_size*(i+1), col_ids].values,
            y_: train_onehot_labels[batch_size*i:batch_size*(i+1), :],
            l2: 0.1})

In [9]:
test_ranges = [
    ('resource', range(test_resource_start, test_severity_start)),
    ('severity', range(test_severity_start, test_volume_start))]

In [10]:
test_col_ids = list_sum(x[1] for x in test_ranges)

In [11]:
prob = y.eval(feed_dict={x: test.iloc[:, test_col_ids]})

In [12]:
prob.shape

(11171, 3)

In [13]:
nn = pd.DataFrame({
    'id': sample.loc[:, 'id'].values, 
    'predict_0': prob[:, 0],
    'predict_1': prob[:, 1],
    'predict_2': prob[:, 2]})

In [14]:
nn.head()

,id,predict_0,predict_1,predict_2
0,11066,0.618390,0.223752,0.157857
1,18000,0.485847,0.300813,0.213340
2,16964,0.618390,0.223752,0.157857
3,4795,0.435015,0.324048,0.240937
4,3392,0.530771,0.274856,0.194373


In [ ]:
nn.to_csv('neural_network.csv', index=False)